In [1]:
'''I have made use of Python NLTK in this code. Please install the  required packages : nltk,pickle. before 
testing this code. Apart from that i have made use of an open source text dataset for training 
my model. 
I have attached the same file. Please extract the same and to run the code in your
system please make sure the repo is in same path as your code file or you can explicitly provide the 
path of repo in the code

Please extract the given folder along with its fill its accurate path in your development environment for proper working of 
the code.
'''

'I have made use of Python NLTK in this code. Please install the  required packages : nltk,pickle. before \ntesting this code. Apart from that i have made use of an open source text dataset for training \nmy model. \nI have attached the same file. Please extract the same and to run the code in your\nsystem please make sure the repo is in same path as your code file or you can explicitly provide the \npath of repo in the code\n\nPlease extract the given folder along with its fill its accurate path in your development environment for proper working of \nthe code.\n'

In [3]:
# Importing the important packages for NLP purposes
import string
from nltk.stem.snowball import SnowballStemmer
import string
import os
from nltk.stem.snowball import SnowballStemmer
import nltk.corpus
import string
import pickle
from collections import Iterable
from nltk.tag import ClassifierBasedTagger
from nltk.chunk import ChunkParserI
 

# here history contains  the previous predicted tags by model
# index is the index of token whose tag has to be predicted
def features(tokens, index, history):
   
 
    # init the stemmer
    # The goal of both stemming and lemmatization is to reduce inflectional forms and sometimes derivationally related forms of a word to a common base form. For instance:
    # am, are, is ----> be 
    # car, cars, car's, cars' ------> car
    # Means stemmer is used to clean the input text
    
    # We are initialisng our stemmer for english language
    stemmer = SnowballStemmer('english')
 
    # Padding of text
    tokens = [('[START2]', '[START2]'), ('[START1]', '[START1]')] + list(tokens) + [('[END1]', '[END1]'), ('[END2]', '[END2]')]
    history = ['[START2]', '[START1]'] + list(history)
 
    # Shifting the index so as to accomodae the padding
    index += 2
 
    word, pos = tokens[index]
    prevword, prevpos = tokens[index - 1]
    prevprevword, prevprevpos = tokens[index - 2]
    nextword, nextpos = tokens[index + 1]
    nextnextword, nextnextpos = tokens[index + 2]
    previob = history[index - 1]
    contains_dash = '-' in word
    contains_dot = '.' in word
    allascii = all([True for c in word if c in string.ascii_lowercase])
 
    allcaps = word == word.capitalize()
    capitalized = word[0] in string.ascii_uppercase
 
    prevallcaps = prevword == prevword.capitalize()
    prevcapitalized = prevword[0] in string.ascii_uppercase
 
    nextallcaps = prevword == prevword.capitalize()
    nextcapitalized = prevword[0] in string.ascii_uppercase
 
    return {
        'word': word,
        'lemma': stemmer.stem(word),
        'pos': pos,
        'all-ascii': allascii,
 
        'next-word': nextword,
        'next-lemma': stemmer.stem(nextword),
        'next-pos': nextpos,
 
        'next-next-word': nextnextword,
        'nextnextpos': nextnextpos,
 
        'prev-word': prevword,
        'prev-lemma': stemmer.stem(prevword),
        'prev-pos': prevpos,
 
        'prev-prev-word': prevprevword,
        'prev-prev-pos': prevprevpos,
 
        'prev-iob': previob,
 
        'contains-dash': contains_dash,
        'contains-dot': contains_dot,
 
        'all-caps': allcaps,
        'capitalized': capitalized,
 
        'prev-all-caps': prevallcaps,
        'prev-capitalized': prevcapitalized,
 
        'next-all-caps': nextallcaps,
        'next-capitalized': nextcapitalized,
    }

In [4]:
# Readng the repository of words
#  Further cleansing of the text
corps_rot = "gmb-2.2.0"
def to_conll_iob(annotated_sentence):
    
    proper_iob_tokens = []
    for idx, annotated_token in enumerate(annotated_sentence):
        tag, word, ner = annotated_token
 
        if ner != 'O':
            if idx == 0:
                ner = "B-" + ner
            elif annotated_sentence[idx - 1][2] == ner:
                ner = "I-" + ner
            else:
                ner = "B-" + ner
        proper_iob_tokens.append((tag, word, ner))
    return proper_iob_tokens
 
 
def read_gmb(corps_rot):
    for root, dirs, files in os.walk(corps_rot):
        for filename in files:
            if filename.endswith(".tags"):
                with open(os.path.join(root, filename), 'rb') as file_handle:
                    file_content = file_handle.read().decode('utf-8').strip()
                    annotated_sentences = file_content.split('\n\n')
                    for annotated_sentence in annotated_sentences:
                        annotated_tokens = [seq for seq in annotated_sentence.split('\n') if seq]
 
                        standard_form_tokens = []
 
                        for idx, annotated_token in enumerate(annotated_tokens):
                            annotations = annotated_token.split('\t')
                            word, tag, ner = annotations[0], annotations[1], annotations[3]
 
                            if ner != 'O':
                                ner = ner.split('-')[0]
 
                            if tag in ('LQU', 'RQU'):   # Make it NLTK compatible
                                tag = "``"
 
                            standard_form_tokens.append((word, tag, ner))
 
                        conll_tokens = to_conll_iob(standard_form_tokens)
 
                        # We are making the NLTK classifier compatible with input text                        
                        yield [((w, t), iob) for w, t, iob in conll_tokens]
 
reader = read_gmb(corps_rot)

print reader.next()


[((u'A', u'DT'), u'O'), ((u'MAN', u'NN'), u'O'), ((u'had', u'VBD'), u'O'), ((u'two', u'CD'), u'O'), ((u'dogs', u'NNS'), u'O'), ((u':', u'IN'), u'O'), ((u'a', u'DT'), u'O'), ((u'Hound', u'NN'), u'O'), ((u',', u','), u'O'), ((u'trained', u'VBN'), u'O'), ((u'to', u'TO'), u'O'), ((u'assist', u'VB'), u'O'), ((u'him', u'PRP'), u'O'), ((u'in', u'IN'), u'O'), ((u'his', u'PRP$'), u'O'), ((u'sports', u'NNS'), u'O'), ((u',', u','), u'O'), ((u'and', u'CC'), u'O'), ((u'a', u'DT'), u'O'), ((u'Housedog', u'NN'), u'O'), ((u',', u','), u'O'), ((u'taught', u'VBD'), u'O'), ((u'to', u'TO'), u'O'), ((u'watch', u'VB'), u'O'), ((u'the', u'DT'), u'O'), ((u'house', u'NN'), u'O'), ((u'.', u'.'), u'O')]


In [5]:
# Chunkparser1 contains the training data
class NerChunker(ChunkParserI):
    def __init__(self, train_sents,**varss):
        assert isinstance(train_sents, Iterable)
 
        self.feature_detector = features
        self.tagger = ClassifierBasedTagger(
            train=train_sents,
            feature_detector=features,
            **varss)
 
    def parse(self, tagged_sent):
        chunks = self.tagger.tag(tagged_sent)
 
        
        # Transforming the result to prefferd list         
        iob_triplets = [(w, t, c) for ((w, t), c) in chunks]
 
        # Transform the list of triplets to nltk.Tree format
        return conlltags2tree(iob_triplets)

In [6]:
reader = read_gmb(corps_rot)
data = list(reader)
training_samples = data[:int(len(data) * 0.9)]
test_samples = data[int(len(data) * 0.9):]
# printing the training_samples 
training_samples

[[((u'A', u'DT'), u'O'),
  ((u'MAN', u'NN'), u'O'),
  ((u'had', u'VBD'), u'O'),
  ((u'two', u'CD'), u'O'),
  ((u'dogs', u'NNS'), u'O'),
  ((u':', u'IN'), u'O'),
  ((u'a', u'DT'), u'O'),
  ((u'Hound', u'NN'), u'O'),
  ((u',', u','), u'O'),
  ((u'trained', u'VBN'), u'O'),
  ((u'to', u'TO'), u'O'),
  ((u'assist', u'VB'), u'O'),
  ((u'him', u'PRP'), u'O'),
  ((u'in', u'IN'), u'O'),
  ((u'his', u'PRP$'), u'O'),
  ((u'sports', u'NNS'), u'O'),
  ((u',', u','), u'O'),
  ((u'and', u'CC'), u'O'),
  ((u'a', u'DT'), u'O'),
  ((u'Housedog', u'NN'), u'O'),
  ((u',', u','), u'O'),
  ((u'taught', u'VBD'), u'O'),
  ((u'to', u'TO'), u'O'),
  ((u'watch', u'VB'), u'O'),
  ((u'the', u'DT'), u'O'),
  ((u'house', u'NN'), u'O'),
  ((u'.', u'.'), u'O')],
 [((u'When', u'WRB'), u'O'),
  ((u'he', u'PRP'), u'O'),
  ((u'returned', u'VBD'), u'O'),
  ((u'home', u'NN'), u'O'),
  ((u'after', u'IN'), u'O'),
  ((u'a', u'DT'), u'O'),
  ((u'good', u'JJ'), u'O'),
  ((u'day', u'NN'), u'O'),
  ((u"'s", u'POS'), u'O'),
  ((u's

In [7]:
import nltk.tag 
from nltk.chunk.util import conlltags2tree

chunker = NerChunker(training_samples[2000:])

In [8]:
from nltk import pos_tag, word_tokenize
#Testing our Classifier on our input text
print chunker.parse(pos_tag(word_tokenize('''A treaty committing the United 
                                          States and Russia to keep their long-range nuclear
                                          arsenals at the lowest levels since early in the Cold War
                                          goes into full effect on Monday. When it was signed eight 
                                          go, President Barack Obama expressed hope that it would be 
                                          a small first step toward deeper reductions, and ultimately
                                          a world without nuclear weapons.
                                            A new nuclear policy issued by the Trump administration on Friday, 
                                          which vows to counter a rush by the Russians to modernize their 
                                          forces even while staying within the treaty limits, is touching off a 
                                          new kind of nuclear arms race. This one is based less on numbers of
                                          weapons and more on novel tactics and technologies, meant to outwit 
                                          and outmaneuver the other side.''')))

# In the output geo means geographical tag and the info about further tags is given on next cell

(S
  A/DT
  treaty/NN
  committing/VBG
  the/DT
  (geo United/NNP States/NNPS)
  and/CC
  (geo Russia/NNP)
  to/TO
  keep/VB
  their/PRP$
  long-range/JJ
  nuclear/JJ
  arsenals/NNS
  at/IN
  the/DT
  lowest/JJS
  levels/NNS
  since/IN
  early/RB
  in/IN
  the/DT
  (org Cold/NNP War/NNP)
  goes/VBZ
  into/IN
  full/JJ
  effect/NN
  on/IN
  (tim Monday/NNP)
  ./.
  When/WRB
  it/PRP
  was/VBD
  signed/VBN
  eight/CD
  go/NNS
  ,/,
  (per President/NNP Barack/NNP Obama/NNP)
  expressed/VBD
  hope/NN
  that/IN
  it/PRP
  would/MD
  be/VB
  a/DT
  small/JJ
  first/JJ
  step/NN
  toward/IN
  deeper/JJR
  reductions/NNS
  ,/,
  and/CC
  ultimately/RB
  a/DT
  world/NN
  without/IN
  nuclear/JJ
  weapons/NNS
  ./.
  A/DT
  new/JJ
  nuclear/JJ
  policy/NN
  issued/VBN
  by/IN
  the/DT
  (org Trump/NNP administration/NN)
  on/IN
  (tim Friday/NNP)
  ,/,
  which/WDT
  vows/VBZ
  to/TO
  counter/VB
  a/DT
  rush/NN
  by/IN
  the/DT
  (gpe Russians/NNPS)
  to/TO
  modernize/VB
  their/PRP$
  force

In [ ]:
'''In the above output the meaning of different tags is given by


        CC Coordinating conjunction
        CD Cardinal number
        DT Determiner
        EX Existential there
        FW Foreign word
        IN Preposition or subordinating conjunction
        JJ Adjective
        JJR Adjective, comparative
        JJS Adjective, superlative
        LS List item marker
        MD Modal
        NN Noun, singular or mass
        NNS Noun, plural
        NNP Proper noun, singular
        NNPS Proper noun, plural
        PDT Predeterminer
        POS Possessive ending
        PRP Personal pronoun
        PRP$ Possessive pronoun
        RB Adverb
        RBR Adverb, comparative
        RBS Adverb, superlative
        RP Particle
        SYM Symbol
        TO to
        UH Interjection
        VB Verb, base form
        VBD Verb, past tense
        VBG Verb, gerund or present participle
        VBN Verb, past participle
        VBP Verb, non­3rd person singular present
        VBZ Verb, 3rd person singular present
        WDT Wh­determiner
        WP Wh­pronoun
        WP$ Possessive wh­pronoun
        WRB Wh­adverb


'''